In [124]:
import numpy as np
import imutils
import cv2 as cv

In [125]:
class Stitcher:
	def __init__(self):
		self.isv3 = imutils.is_cv3(or_better = True)

	def stitch(self, images, ratio = 0.75, reprojThresh = 4.0, showMatches = False,matcher_name="FlannBased"):
		(imageB, imageA) = images
		#Calculate the keypoints and the respective descriptors
		(interestPointsA, featuresA) = self.detectAndDescribe(imageA)
		(interestPointsB, featuresB) = self.detectAndDescribe(imageB)
		# match features between the two images
		M = self.matchInterestPoints(interestPointsA, interestPointsB, featuresA, featuresB, ratio, reprojThresh,matcher_name)
		if M is None:
			return None
		#If enough interest points are present
		# apply a perspective warp to stitch the images together
		(matches, H, status) = M
		stitchedImage = cv.warpPerspective(imageA, H, (imageA.shape[1] + imageB.shape[1], imageB.shape[0]))
		stitchedImage[0:imageB.shape[0], 0:imageB.shape[1]] = imageB
		# If the user requested the matches to be seen the visualization image is returned
		if showMatches:
			visualization = self.drawMatches(imageA, imageB, interestPointsA, interestPointsB, matches, status)
			return (stitchedImage, visualization)
		return stitchedImage
	
	#This function extracts the keypoints and their respective descriptors using SIFT algorithm
	def detectAndDescribe(self, image):
		gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
		if self.isv3:
				descriptor = cv.SIFT_create()
				(kps, features) = descriptor.detectAndCompute(gray, None)
		else:
				# detect interestpoints in the image
				detector = cv.FeatureDetector_create("SIFT")
				kps = detector.detect(gray)
				# extract features from the image
				extractor = cv.DescriptorExtractor_create("SIFT")
				(kps, features) = extractor.compute(gray, kps)
		# convert the interestpoints from interestPoint objects to NumPy arrays
		kps = np.float32([kp.pt for kp in kps])
		# return a tuple of interestpoints and features
		return (kps, features)
		
	##Function to get the Homography matrix of the two images
	def matchInterestPoints(self, interestPointsA, interestPointsB, featuresA, featuresB, ratio, reprojThresh,matcher_name):
		# compute the raw matches and initialize the list of actual matches
		matcher = cv.DescriptorMatcher_create(matcher_name)
		rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
		matches = []
		for m in rawMatches:
			# ensure the distance is within a certain ratio of each other (i.e. Lowe's ratio test)
			if len(m) == 2 and m[0].distance < m[1].distance * ratio:
				matches.append((m[0].trainIdx, m[0].queryIdx))
		# computing a homography requires at least 4 matches
		if len(matches) > 4:
			# construct the two sets of points
			ptsA = np.float32([interestPointsA[i] for (_, i) in matches])
			ptsB = np.float32([interestPointsB[i] for (i, _) in matches])
			# compute the homography between the two sets of points
			(H, status) = cv.findHomography(ptsA, ptsB, cv.RANSAC,
				reprojThresh)
			# return the matches along with the homograpy matrix and status of each matched point
			return (matches, H, status)
		return None

	def drawMatches(self, imageA, imageB, interestPointsA, interestPointsB, matches, status):
		# initialize the output visualization image
		(hA, wA), (hB, wB) = imageA.shape[:2], imageB.shape[:2]
		visualization = np.zeros((max(hA, hB), wA + wB, 3), dtype = "uint8")
		visualization[0:hA, 0:wA], visualization[0:hB, wA:] = imageA, imageB
		for ((trainIdx, queryIdx), s) in zip(matches, status):
			# only process the match if the interest point was successfully matched
			if s == 1:
				# draw the match
				ptA = (int(interestPointsA[queryIdx][0]), int(interestPointsA[queryIdx][1]))
				ptB = (int(interestPointsB[trainIdx][0]) + wA, int(interestPointsB[trainIdx][1]))
				cv.line(visualization, ptA, ptB, (70, 90, 255), 1)
		return visualization

In [126]:

def create_img(imagepathA,imagepathB,output_path,widthA,widthB,height):
    imageA = cv.imread(imagepathA)
    imageB = cv.imread(imagepathB)
    imageA = imutils.resize(imageA, widthA,height)
    imageB = imutils.resize(imageB, widthB,height)

    stitcher = Stitcher()
    (stichedImage, visualization) = stitcher.stitch([imageA, imageB], showMatches = True)
    cv.imwrite("InterestPoints2.jpg", visualization)
    cv.imwrite(output_path, stichedImage)

In [127]:
imagepathA = "../Dataset/Part-A/VS2.jpeg"
imagepathB = "../Dataset/Part-A/VS3.jpeg"
output_path = "VidhanSoudha23Stitched.jpg"
create_img(imagepathA,imagepathB,output_path,900,900,600)



In [128]:
imagepathA = "../Dataset/Part-A/VS1.jpeg"
imagepathB = "VidhanSoudha23Stitched.jpg"
output_path = "VidhanSoudha123Stitched.jpg"
create_img(imagepathA,imagepathB,output_path,900,1600,600)


In [129]:
imagepathA = "../Dataset/Part-A/CB_2.jpeg"
imagepathB = "../Dataset/Part-A/CB_1.jpeg"
output_path = "Cubbon_park_stitched.jpg"

create_img(imagepathA,imagepathB,output_path,600,600,900)

In [130]:
imagepathA = "../Dataset/Part-A/BRL.jpeg"
imagepathB = "../Dataset/Part-A/BRR.jpeg"
output_path = "Brigade_Road_stitched.jpg"

create_img(imagepathA,imagepathB,output_path,800,800,800)